# solve_ivp gives "wobbly" results?

## Purpose
* solve_ivp with RK45 gives some wobbly results can this be improved?

## Methodology
* Run solve_ivp with various settings.
* compare the accelerations.

## Results
Describe and comment the most important results.

## Setup

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.regression import show_pred
from vessel_manoeuvring_models.visualization.plot import track_plot

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_simple_nonlinear  as vmm
from vessel_manoeuvring_models.data.case_1 import ship_parameters, df_parameters, ps, ship_parameters_prime
from vessel_manoeuvring_models.data.transform import transform_to_ship

## Ship parameters

In [ ]:
ship_parameters

## Brix parameters

In [ ]:
mask = df_parameters['prime'].notnull()
index = df_parameters.loc[mask,'prime'].index
coefficients=vmm.simulator.get_all_coefficients(sympy_symbols=False)
missing_coefficients = set(coefficients) - set(index)
missing_coefficients

In [ ]:
mask = df_parameters['prime'].notnull()
df_parameters.loc[mask,'prime']

## Simulate data

In [ ]:
parameters=df_parameters['prime'].copy()

t_ = np.linspace(0,70,1000)
df_ = pd.DataFrame(index=t_)

df_['u'] = 2
df_['v'] = 0
df_['r'] = 0
df_['x0'] = 0
df_['y0'] = 0
df_['psi'] = 0
df_['U'] = np.sqrt(df_['u']**2 + df_['v']**2)
df_['beta'] = -np.arctan2(df_['v'],df_['u'])
df_['thrust'] = 50

df_['delta'] = 0
df_.loc[10:,'delta'] = np.deg2rad(20)

results = {}
for method in ['RK45','Radau','BDF','RK23','DOP853','LSODA']:
    
    result = vmm.simulator.simulate(df_=df_, parameters=parameters, ship_parameters=ship_parameters, 
                                control_keys=['delta','thrust'], primed_parameters=True,
                                prime_system=ps, method=method)
    results[method] = result
 

In [ ]:
results=pd.Series(results)

## Compare

In [ ]:
fig,ax=plt.subplots()
for method,result in results.items():
    result.result.plot(y='u1d', label=method, ax=ax);

ax.set_ylim(results['RK45'].result['u1d'].min(), results['RK45'].result['u1d'].max())

In [ ]:
fig,ax=plt.subplots()
for method,result in results.loc[['Radau','BDF','LSODA']].items():
    result.result.plot(y='u1d', label=method, ax=ax);

ax.set_ylim(results['RK45'].result['u1d'].min(), results['RK45'].result['u1d'].max())

In [ ]:
x,y,z = sp.symbols('x y z')
M = sp.Matrix([sp.sin(x) + y, sp.cos(y) + x, z]) 
M

In [ ]:
M.jacobian([x, y, z])

In [ ]:
eq_acceleration = vmm.simulator.acceleartion_eq.subs([(X_qs,vmm.X_qs_eq.rhs),
                                    (Y_qs,vmm.Y_qs_eq.rhs),
                                    (N_qs,vmm.N_qs_eq.rhs),
                                   ])
subs = {value:key for key,value in p.items()} 
eq_acceleration = eq_acceleration.subs(subs)

jac = eq_acceleration.jacobian([u,v,r])
jac_lambda=lambdify(jac)

In [ ]:
jac_lambda